# 🧮 Validación Simbiótica ∞³: Hipótesis de Riemann via Sistemas Adélicos

## Notebook de Validación Numérica

**Autor**: José Manuel Mota Burruezo  
**Fecha**: Septiembre 2025  
**Repositorio**: [motanova84/-jmmotaburr-riemann-adelic](https://github.com/motanova84/-jmmotaburr-riemann-adelic)

Este notebook implementa la validación numérica rigurosa de la fórmula explícita tipo Weil para la función canónica $D(s)$ construida via flujos adélicos, sin usar el producto de Euler de $\zeta(s)$.

### 📐 Fórmula a Validar
$$\sum_p \sum_{k=1}^K \log(p) \cdot f(k \log p) + A_\infty(f) = \sum_\gamma \mathfrak{Re}[\hat{f}(\gamma)]$$

**Donde:**
- Lado izquierdo: Contribución aritmética (primos + arquimediana)
- Lado derecho: Suma sobre ceros no triviales $\gamma$ de $\zeta(s)$
- $\hat{f}(s)$: Transformada de Mellin de $f(u)$

In [ ]:
# Configuración inicial: Importar bibliotecas y configurar precisión
import mpmath as mp
import sympy as sp
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import os

# Agregar directorio padre al path para importar utils
sys.path.append('..')
try:
    from utils.mellin import truncated_gaussian, mellin_transform
    from utils.riemann_tools import t_to_n, load_zeros_near_t
    print("✅ Utils importadas exitosamente")
except ImportError:
    print("⚠️ Utils no disponibles, usando definiciones locales")

# Configurar precisión: 50 dígitos decimales para validación rigurosa
mp.mp.dps = 50

print(f"🔬 Configuración simbiótica ∞³ iniciada")
print(f"📊 Precisión aritmética: {mp.mp.dps} dígitos decimales")
print(f"🗓️ Timestamp: {datetime.now().isoformat()}")

## 📝 Definición de Funciones Test

Implementamos múltiples funciones test con soporte compacto para validar la convergencia numérica:

In [ ]:
# Funciones test con soporte compacto
def f1_gaussian_truncated(u, a=3.0, sigma=1.0):
    """Gaussiana truncada: exp(-u²/2σ²) para |u| ≤ a"""
    return mp.exp(-u**2/(2*sigma**2)) if abs(u) <= a else mp.mpf(0)

def f2_double_exponential(u, a=2.0):
    """Exponencial doble: exp(-u²) para |u| ≤ a"""
    return mp.exp(-u**2) if abs(u) <= a else mp.mpf(0)

def f3_polynomial_compact(u, a=2.0):
    """Polinomial compacto: (1 - u²/4)² para |u| ≤ a"""
    return (1 - u**2/4)**2 if abs(u) <= a else mp.mpf(0)

# Lista de funciones test para validación sistemática
test_functions = [
    ('f1_gaussian', f1_gaussian_truncated, 3.0),
    ('f2_exponential', f2_double_exponential, 2.0),
    ('f3_polynomial', f3_polynomial_compact, 2.0)
]

print(f"✅ {len(test_functions)} funciones test definidas")
for name, func, lim in test_functions:
    print(f"  - {name}: soporte [-{lim}, {lim}]")

## ⚙️ Implementación de Componentes de la Fórmula Explícita

In [ ]:
def transformada_mellin(f, s, lim_u=5.0):
    """Transformada de Mellin numérica: ∫ f(u) e^{su} du"""
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim_u, lim_u], maxdegree=20)

def suma_sobre_primos(f, P=1000, K=5):
    """Suma sobre potencias de primos: Σ_p Σ_k log(p) * f(k*log(p))"""
    total = mp.mpf('0')
    primes = list(sp.primerange(2, P))
    print(f"📊 Procesando {len(primes)} primos hasta P={P}, potencias K={K}")
    
    for i, p in enumerate(primes):
        if i % 100 == 0:  # Progress indicator
            print(f"  Progreso: {i}/{len(primes)} primos procesados")
        lp = mp.log(p)
        for k in range(1, K+1):
            total += lp * f(k * lp)
    return total

def contribucion_arquimediana(f, sigma0=2.0, T=50, lim_u=3):
    """Contribución arquimediana A_∞(f)"""
    print(f"🏛️ Calculando contribución arquimediana: σ₀={sigma0}, T={T}")
    
    def integrand(t):
        s = mp.mpc(sigma0, t)
        kernel = mp.digamma(s/2) - mp.log(mp.pi)
        return kernel * transformada_mellin(f, s, lim_u)
    
    integral = mp.quad(integrand, [-T, T], maxdegree=10) / (2 * mp.pi)
    
    # Término residual en s=1
    residue_term = transformada_mellin(f, mp.mpf(1), lim_u) / mp.mpf(1)
    
    return integral - residue_term

def suma_sobre_ceros(f, N=1000, lim_u=3):
    """Suma sobre ceros no triviales usando mpmath.zetazero"""
    print(f"🎯 Calculando suma sobre {N} ceros de Riemann")
    total = mp.mpf('0')
    
    for n in range(1, N+1):
        if n % 200 == 0:  # Progress indicator
            print(f"  Progreso: {n}/{N} ceros procesados")
        
        # Obtener n-ésimo cero no trivial
        rho = mp.zetazero(n)
        gamma = mp.im(rho)  # Parte imaginaria = altura del cero
        
        # Evaluar transformada de Mellin en iγ
        fhat_val = transformada_mellin(f, 1j * gamma, lim_u)
        total += fhat_val.real
    
    return total

print("✅ Funciones de validación implementadas")

## 🧮 Validación Numérica Principal

Ejecutamos la validación para cada función test y recopilamos resultados:

In [ ]:
# Parámetros de validación
PARAMETROS = {
    'P': 1000,      # Primos hasta ~7919
    'K': 5,         # Potencias por primo
    'T': 50,        # Altura máxima para integración
    'N_zeros': 1000, # Número de ceros de Riemann
    'sigma0': 2.0   # Línea de integración
}

print("🚀 Iniciando validación simbiótica ∞³")
print(f"📋 Parámetros: {PARAMETROS}")
print("="*60)

resultados = []
start_time = datetime.now()

for name, f, lim_u in test_functions:
    print(f"\n🔬 VALIDANDO: {name}")
    print("-" * 40)
    
    func_start = datetime.now()
    
    # Lado aritmético
    print("1️⃣ Calculando suma sobre primos...")
    prime_side = suma_sobre_primos(f, PARAMETROS['P'], PARAMETROS['K'])
    
    print("2️⃣ Calculando contribución arquimediana...")
    arch_side = contribucion_arquimediana(f, PARAMETROS['sigma0'], PARAMETROS['T'], lim_u)
    
    arithmetic_total = prime_side + arch_side
    
    # Lado de ceros
    print("3️⃣ Calculando suma sobre ceros...")
    zero_side = suma_sobre_ceros(f, PARAMETROS['N_zeros'], lim_u)
    
    # Análisis de resultados
    error_absoluto = abs(arithmetic_total - zero_side)
    error_relativo = error_absoluto / abs(arithmetic_total) if arithmetic_total != 0 else mp.inf
    
    func_time = (datetime.now() - func_start).total_seconds()
    
    # Reporte
    print(f"\n📊 RESULTADOS para {name}:")
    print(f"   Lado Aritmético: {arithmetic_total:.15e}")
    print(f"   Lado de Ceros:   {zero_side:.15e}")
    print(f"   Error Absoluto:  {error_absoluto:.6e}")
    print(f"   Error Relativo:  {error_relativo:.6e}")
    print(f"   Tiempo:          {func_time:.2f} segundos")
    
    # Guardar resultado
    resultados.append({
        'function': name,
        'arithmetic_side': float(arithmetic_total),
        'zero_side': float(zero_side),
        'absolute_error': float(error_absoluto),
        'relative_error': float(error_relativo),
        'execution_time': func_time,
        'precision': mp.mp.dps,
        'parameters': PARAMETROS.copy()
    })

total_time = (datetime.now() - start_time).total_seconds()
print(f"\n🏁 VALIDACIÓN COMPLETA")
print(f"⏱️ Tiempo total: {total_time:.2f} segundos")
print("=" * 60)

## 📈 Análisis de Resultados y Conclusiones

In [ ]:
# Crear DataFrame con resultados
df_resultados = pd.DataFrame(resultados)

print("📊 RESUMEN DE VALIDACIÓN SIMBIÓTICA ∞³")
print("=" * 50)

for _, row in df_resultados.iterrows():
    print(f"\n🔸 {row['function']}:")
    print(f"   Error Relativo: {row['relative_error']:.2e}")
    print(f"   Tiempo: {row['execution_time']:.1f}s")
    
    # Evaluación de precisión
    if row['relative_error'] < 1e-10:
        print("   ✅ EXCELENTE precisión numérica")
    elif row['relative_error'] < 1e-6:
        print("   ✅ BUENA precisión numérica")
    elif row['relative_error'] < 1e-3:
        print("   ⚠️ Precisión MODERADA")
    else:
        print("   ❌ Precisión INSUFICIENTE")

print(f"\n🎯 CONCLUSIÓN GENERAL:")
errores_relativos = df_resultados['relative_error']
error_promedio = errores_relativos.mean()
error_maximo = errores_relativos.max()

print(f"   Error Relativo Promedio: {error_promedio:.2e}")
print(f"   Error Relativo Máximo:   {error_maximo:.2e}")

if error_maximo < 1e-8:
    print("   🏆 VALIDACIÓN EXITOSA: Fórmula explícita verificada numéricamente")
    print("   🧮 Flujo simbiótico ∞³ CONFIRMADO")
elif error_maximo < 1e-5:
    print("   ✅ VALIDACIÓN SATISFACTORIA: Evidencia numérica sólida")
else:
    print("   ⚠️ VALIDACIÓN PARCIAL: Se requiere mayor precisión o más términos")

print(f"\n📊 Precisión utilizada: {mp.mp.dps} dígitos decimales")
print(f"🗓️ Timestamp final: {datetime.now().isoformat()}")

## 💾 Guardado de Resultados

In [ ]:
# Guardar resultados en CSV
try:
    output_file = '../data/validation_output.csv'
    
    # Preparar datos para CSV
    csv_data = []
    for result in resultados:
        csv_data.append({
            'timestamp': datetime.now().isoformat(),
            'test_function': result['function'],
            'arithmetic_side': result['arithmetic_side'],
            'zero_side': result['zero_side'],
            'absolute_error': result['absolute_error'],
            'relative_error': result['relative_error'],
            'precision_digits': result['precision'],
            'max_prime': result['parameters']['P'],
            'max_power': result['parameters']['K'],
            'execution_time_seconds': result['execution_time']
        })
    
    # Escribir CSV
    df_csv = pd.DataFrame(csv_data)
    
    # Crear header si el archivo no existe
    write_header = not os.path.exists(output_file)
    
    df_csv.to_csv(output_file, mode='a', header=write_header, index=False)
    print(f"✅ Resultados guardados en: {output_file}")
    
except Exception as e:
    print(f"⚠️ Error guardando resultados: {e}")

print("\n🎊 NOTEBOOK DE VALIDACIÓN SIMBIÓTICA ∞³ COMPLETADO")
print("📜 Para más información: https://github.com/motanova84/-jmmotaburr-riemann-adelic")